# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

<div class="alert alert-block alert-warning">
**Note:** Make sure you have selected a **Python 3** kernel in Workspaces or the hmm-tagger conda environment if you are running the Jupyter server on your own machine.
</div>

In [82]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [85]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [86]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [87]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [88]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [89]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [90]:
tags = [pair[1] for pair in data.stream()]
words = [pair[0] for pair in data.stream()]

In [91]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!

    dd = defaultdict(dict) 

    for pair,count in dict(Counter((zip(sequences_A, sequences_B)))).items():
        dd[pair[0]][pair[1]] = count
    return dd

    raise NotImplementedError

# Calculate C(t_i, w_i)
emission_counts = pair_counts(tags,words)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [92]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently 
# assigned to that word

from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))
    
# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

tags_train = [tag for word, tag in data.training_set.stream()]
words_train = [word for word, tag in data.training_set.stream()]

word_counts = pair_counts(words_train, tags_train)

# TODO: Your code here!
mfc_table={}

for key, value in word_counts.items():
    cnt = Counter(value).most_common(1)
    mfc_table[key] = cnt[0][0]
    

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [93]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [94]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [95]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [96]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.02%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [97]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
    return Counter(sequences)
    raise NotImplementedError

# TODO: call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts(tags_train)  # TODO: YOUR CODE HERE)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [98]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    
    # TODO: Finish this function!
    
    bigrams = list(zip(sequences, sequences[1:]))
    return Counter(bigrams)
    
    raise NotImplementedError

# TODO: call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(tags_train) # TODO: YOUR CODE HERE)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [99]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    # TODO: Finish this function!
    start_list = [entry[0] for entry in sequences]
    return Counter(start_list)
    raise NotImplementedError

# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.training_set.Y)# TODO: YOUR CODE HERE)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [100]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # TODO: Finish this function!
    end_list = [entry[-1] for entry in sequences]
    return Counter(end_list)
    raise NotImplementedError

# TODO: Calculate the count of each tag ending a sequence
tag_ends = ending_counts(data.training_set.Y)# TODO: YOUR CODE HERE)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [128]:
# the list of 12 tags

tags_list = list(set(tags_train))

12

In [127]:
# c(t) = number of times t shows as a tag in the training set

for tag in tags_list: 
    c_t = tag_unigrams[tag]
    print(c_t)

109671
115808
117757
66754
30537
23906
146161
44877
1094
220632
39383
11878


In [141]:
#c(w, t) = number of times the pair (word, tag) appears in the training set

dict_cwt = Counter(zip(words_train,tags_train))
for (word, tag) in dict_cwt:
    print(dict_cwt[(word,tag)])


3
1
1
3
2
1
2
1
3
1
6
6
1
4
1
1
1
6
5
1
1
1
18
1
1434
1
1
1
2
1
1
1
1
1
1
1
4
59
1
2
7
2
1
5
2
1
1
4
1
1
28
13
2
1
1
1
3
1
2
2
1
2
3
1
1
1
6
3
3
2
1
1
1
1
1
11
1
1
25
1
16
1
10
1
1
1
2
1
2
1
26
2
2
1
2
1
2
1
1
1
3
1
43
1
4
10
2
1
1
1
6
13
1
18
1
12
1
1
13
1
1
2
5
1
1
1
3
1
1
1
3
2
1
1
2
1
1
1
4
1
4
1
1
1
4
7
1
48
1
3
3
2
2844
1
11
1
4
2
1
10
8
1
1
2
79
1
124
2
1
1
1
1
1
1
1
1
6
2
1
3
1
31
1
1
1
40
1
3
1
3
2
1
15
1
1
5
1
1
9
1
6
3
2
1
7
2
1
12
1
3
1
3
60
2
6
9
1
2
8
2
1
3
1
11
6
34
1
2
3
4
1
9
1
1
1
1
15
3
11
1
8
1
29
1
1
22
5
2
3
1
1
7
1
2
2
3
4
6
1
4
2
1
1
3
25
2
1
248
2
39
1
3
1
3
10
1
1
3
1
4
5
2
2
1
2
17
1
19
1
2
1
9
1
1
1
1
9
1
1
2
4
3
1
5
8
2
3
3
1
30
1
1
13
11
8
1
1
1
1
4
10
14
2
1
12
38
2
54
38
1
13
1
3
2
10
6
3
1
1
3
181
1
1
1
1
1
1
2
1
1
1
1
2
1
1
10
2
1
2
1
17
2
12
2
1
2
1
2
1
45
1
3
1
2
1
2
8
1
3
4
3
1
3
2
2
2
1
6
1
15
3
6
6
1
1
5
3
15
1
1
5
9
10
3
1
7
1
2
2
5
5
5
3
2
1
1
17
1
1
3
3
3
2
1
2
3
8
1
5
1
13
1
2
1
2
1
2
5
64
1
1
7
18
54
1
1
13
3
1
13
1
1
1
3
14
6
1
1
14
1
11
2
3

92
8
1
1
8
10
1
1
13
1
2
1
1
3
1
1
1
12
30
1
1
4
1
3
10
2
1
7
14
5
5
1
12
8
5
2
1
1
2
1
1
1
3
38
1
1
1
2
1
1
8
12
4
1
1
5
12
2
1
31
4
1
1
2
1
1
2
1
55
2
3
1
53
2
6
212
5
1
1
2
1
5
1
1
3
1
3
1
3
1
3
1
4
2
1
2
1
5
1
1
2
1
1
15
1
3
2
7
1
4
2
1
3
1
1
1
1
84
20
19
3
18
1
1
1
1
1
1
1
1
1
11
9
2
32
3
8
2
44
1
2
13
1
5
3
1
1
10
10
1
2
1
6
1
4
1
7
9
2
1
1
3
1
1
1
2
18
4
1
1
1
1
1
20
1
1
1
45
13
2
7
15
1
7
5
1
1
1
1
2
1
2
4
11
17
2
1
7
1
8
5
1
2
1
1
21
1
62
12
156
3
4
38
1
1
3
28
1
56
6
8
4
1
1
2
1
1
9
2
3
1
1
1
12
2
4
4
19
1
1
1
4
3
68
2
1
3
1
2
6
6
6
55
7
1
1
3
1
3
1
1
3
2
2
6
1
2
8
7
2
1
1
1
2
1
1
5
2
1
1
3
9
1
3
8
2
1
4
6
1
2
8
1
1
1
2
2
1
3
1
32
129
1
2
1
2
1
33
6
2
1
2
1
8
4
4
1
1
1
3
1
1
2
2
1
1
2
4
1
11
8
1
3
13
5
2
158
1
1
3
1
20
2
3
4
1
33
2
9
1
1
4
6
1
2
1
2
1
1
1
1
1
2
4
3
3
1
7
1
1
1
4
19
2
15
1
2
4
1
72
1
1
1
42
1
1
1
26
1
1
2
1
1
2
1
1
4
3
1
3
1
1
7
10
1
1
1
1
1
1
1
87
1
1
1
21
5
16
1
22
1
1
1
6
3
6
2
1
1
1
1
4
10
6
89
13
1
28
12
1
11
3
5
1
1
2
2
1
5
2
1
1
1
1
8
6
6
1
2
1
8
1
5
9


2
1
4
21
1
8
1
1
8
1
1
2
3
12
5
2
1
3
3
3
1
2
4
3
1
1
2
5
2
1
1
1
1
1
9
1
1
1
2
13
38
13
1
3
2
15
1
3
1
2
2
1
1
7
1
1
1
1
10
6
2
35
12
1
3
5
20
1
48
3
10
1
1
1
1
3
1
2
1
1
1
1
4
1
1
3
2
1
2
1
5
5
4
2
2
1
1
2
2
6
3
1
84
19
1
1
28
5
4
3
1
1
6
2
48
3
11
1
1
1
1
2
2
2
1
2
1
1
3
7
1
1
2
1
1
1
2
1
2
1
1
2
1
4
9
2
1
2
1
1
44
9
2
1
3
38
1
1
1
1
1
2
4
1
3
18
2
2
1
1
1
1
1
3
1
24
1
2
1
1
1
1
2
3
1
1
1
5
1
2
52
1
1
5
2
1
1
3
2
1
2
1
1
1
67
1
1
7
1
1
1
1
1
1
1
2
2
1
2
1
1
4
1
1
3
1
3
5
4
14
17
2
64
1
1
1
1
2
1
34
1
1
2
2
1
3
1
1
1
24
8
3
103
3
1
8
5
4
5
1
1
1
1
1
3
3
1
1
1
7
4
1
2
3
1
2
1
2
3
13
1
3
7
5
2
3
1
17
2
1
1
3
1
2
4
1
1
2
1
2
1
5
1
2
1
1
1
2
6
4
2
4
1
1
1
3
2
5
1
2
16
1
54
1
1
1
4
1
1
29
1
1
1
1
35
1
1
2
4
14
1
1
1
1
4
1
15
1
1
3
4
11
17
1
7
1
32
29
1
3
1
1
2
2
162
1
3
10
4
2
3
4
13
2
5
2
100
1
1
12
3
3
1
1
2
68
5
1
20
7
4
6
1
1
2
3
33
3
1
1
6
6
5
1
4
3
12
5
5
1
1
11
1
1
1
1
15
2
1
1
1
8
1
1
8
1
1
1
3
1
10
36
1
1
2
1
2
1
12
2
1
1
2
3
1
4
1
2
65
2
1
11
2
1
1
2
4
1
6
1
1
2
1
1
1
6
1
3
1
8


1
1
3
3
1
3
3
4
1
4
1
1
3
4
1
26
1
1
2
3
1
29
3
2
15
8
3
1
7
86
1
1
3
1
3
2
7
2
2
1
1
15
1
3
16
1
35
8
1
3
1
1
1
1
10
1
1
1
2
1
2
1
1
2
1
21
2
1
1
2
1
56
11
1
1
1
1
1
2
2
2
1
2
4
1
1
1
1
3
1
1
5
1
4
3
10
1
1
35
2
1
1
2
1
2
3
28
10
4
2
1
7
3
2
1
1
23
12
45
3
1
2
1
3
1
1
1
4
5
2
2
7
4
3
5
2
31
4
8
20
1
1
12
1
1
2
1
1
1
20
11
9
1
6
2
1
6
2
9
1
1
3
2
1
2
2
1
2
2
1
1
2
1
6
3
1
7
1
1
1
117
56
1
7
8
15
3
1
421
2
1
1
3
2
1
1
2
6
2
3
1
1
5
1
11
30
1
5
1
44
12
12
30
10
1
1
1
5
37
1
1
3
1
1
1
3
1
10
3
1
1
1
8
6
4
1
2
1
1
1
6
2
18
1
5
2
10
1
1
1
12
1
1
2
1
6
2
2
2
1
12
3
89
1
1
1
2
1
65
2
1
1
6
1
1
2
2
3
1
1
3
10
1
1
20
3
1
713
1
1
6
7
1
1
1
1
4
3
1
7
1
3
2
25
1
1
1
14
1
1
1
1
2
1
3
1
9
4
11
5
1
1
1
13
3
4
3
1
1
1
22
4
7
3
1
1
1
3
1
2
7
1
1
6
1
1
13
4
2
22
3
4
1
6
7
1
1
1
1
1
1
1
2
3
1
2
1
2
10
2
23
2
33
1
1
5
2
1
2
1
1
4
2
1
2
14
39
3
6
23
3
1
4
2
3
1
1
1
3
1
3
1
19
6
4
1
4
2
3
4
1
5
1
13
3
16
48
7
1
1
4
1
11
158
2
8
2
1
1
2
16
1
160
1
1
1
1
1
2
1
1
1
6
1
1
1
1
1
1
2
5
2
1
5
1
2
1
2
6
1
6
33
7
1


1
1
2
1
17
1
1
1
7
1
1
3
23
100
1
5
2
3
1
1
1
1
1
2
2
2
1
1
1
2
1
1
4
1
1
1
1
6
1
1
1
1
2
4
1
1
1
4
1
1
18
1
2
1
1
9
1
1
7
1
44
1
1
2
1
1
6
4
1
1
1
2
1
8
1
29
9
123
1
29
18
3
4
5
4
3
3
1
1
2
7
7
6
3
2
2
2
1
26
11
4
1
2
1
1
1
4
2
3
1
1
1
23
1
1
1
2
1
1
191
1
4
1
1
1
1
250
1
1
3
3
3
5
1
1
1
10
1
1
1
4
2
1
3
11
6
1
13
1
7
2
5
7
1
2
2
33
1
8
2
2
1
1
1
6
6
1
2
1
3
1
4
1
1
4
4
2
2
2
4
2
10
1
1
1
1
1
1
1
1
2
6
2
1
1
1
5
1
1
2
2
1
1
13
1
1
1
1
1
2
27
2
6
5
10
11
2
1
35
4
1
3
26
1
1
12
1
1
2
1
2
1
2
1
1
2
10
2
2
1
1
1
1
1
2
21
2
1
2
1
3
5
1
2
1
18
1
20
1
2
52
1
2
1
1
851
1
1
1
1
1
32
1
1
1
1
1
1
1
1
1
2
2
1
1
5
1
2
12
1
1
3
3
1
1
2
1
28
1
1
57
1
1
1
35
7
1
3
22
1
1
2
10
1
1
1
1
1
1
10
5
4
3
13
1
11
2
11
2
1
5
4
26
1
5
1
4
12
1
2
1
1
1
1
1
1
1
1
1
1
1
1
4
1
6
5
3
2
22
1
3
1
4
3
4
1
5
2
1
1
42
7
2
2
4
1
3
5
2
1
1
1
4
1
8
1
1
1
2
1
1
2
1
7
2
2
9
2
13
23
1
1
9
32
10
43
1
1
1
1
1
1
1
1
2
1
1
1
9
1
3
1
1
1
10
3
1
2
1
4
10
3
6
1
1
1
1
1
1
1
6
1
4
5
62
1
7
2
1
1
2
4
171
1
1
1
1
1
1
16
1
2
4
44
1
1
3
47

...

3
5
31
12
1
23
1
1
1
7
3
1
1
1
1
3
5
3
1
1
3
1
7
1
3
1
8
8
6
6
123
1
1
7
1
1
1
1
7
1
1
1
1
1
1
2
6
1
23
3
1
1
1
1
1
2
2
2
6
1
2
1
2
2
2
6
2
5
3
3
1
1
3
1
3
4
11
1
6
1
1
14
127
3
5
18
1
1
11
1
3
34
1
1
3
1
1
1
7
15
1
1
2
2
3
1
7
1
1
1
1
12
22
2
3
5
1
3
1
78
1
27
1
1
1
14
1
9
4
1
1
1
1
1
6
1
4
2
3
1
2
1
1
1
2
2
5
10
1
1
4
1
2
1
1
1
3
4
1
1
1
7
4140
1
1
1
2
1
1
12
1
1
1
2
1
1
2
1
32
1
1
1
3
1
3
16
4
3
1
20
2
2
3
1
4
1
12
1
1
1
1
8
1
2
1
1
1
20
1
40
1
3
9
29
37
1
1
1
3
17
2
4
1
1
1
161
47
1
78
1
1
1
1
8
2
1
2
1
1
1
1
1
2
2
1
1
1
1
1
3
5
27
3
10
7
1
2
1
2
3
1
203
94
3
2
1
2
10
10
1
4
2
64
2
1
2
23
24
1
1
8
12
1
10
1
5
1
5
1
5
4
1
26
1
19
1
1
1
1
2
1
1
47
1
4
1
1
3
3
1
7
1
3
3
2
5
13
1
2
5
1
1
1
20
36
2
1
1
1
2
1
1
9
2
13
17
6
2
7
3
1
1
1
1
12
2
4
1
2
1
1
3
1
2
1
12
3
21
1
3
3
6
75
2
7
6
2
1
29
4
3
1
47
2
2
1
2
1
2
1
1
3
7
3
1
2
8
1
1
4
4
15
55
1
1
1
1
1
1
1
1
1
2
11
7
2
1
1
3
3
1
1
1
26
1
2
7
1
1
1
1
1
1
3
1
1
1
1
5
1
1
2
1
17
2
2
2
11
1
10
1
1
2
12
1
28
1
2
1
1
1
4
2
2
67
2
1
1
3
3
170
1
1

1
1
2
3
1
1
8
1
6
1
1
6
1
12
4
2
1
2
1
9
1
2
1
1
2
1
6
1
118
49
7
4
12
1
98
16
2
5
33
10
3
1
6
1
2
1
268
3
1
1
36
10
4
4
7
7
1
1
1
4
1
1
1
9
8
1
1
1
1
2
1
1
3
1
1
4
1
1
5
3
1
1
1
1
1
1
1
1
1
1
1
1
7
6
1
12
2
2
6
4
1
1
1
1
1
3
1
98
1
2
1
3
1
3
20
1
2
1
11
5
1
1
2
1
1
1
1
1
8
3
1
1
1
1
2
5
1
45
6
2
1
1
12
1
1
11
1
1
6
1
1
1
6
5
1
1
1
24
1
38
6
1
3
1
1
2
1
1
1
1
1
12
1
3
47
6
1
88
1
2
1
1
21
1
1
1
6
38
1
3
1
1
3
1
9
1
36
1
2
4
2
5
1
1
19
6
2
5
1
1
78
2
158
9
1
2
1
1
5
3
3
3
1
1
1
1
1
5
3
21
1
1
1
4
1
1
3
8
1
2
15
1
1
1
3
6
3
1
2
1
2
9
1
1
4
1
1
1
10
1
22
1
1
3
1
3
2
1
1
2
101
72
3
3
1
13
3
3
1
1
4
1
1
2
4
8
2
64
7
1
2
4
2
1
8
1
1
18
5
1
1
1
1
1
1
11
1
4
11
1
18
1
1
12
1
3
7
29
2
3
1
1
1
2
1
8
1
2
9
2
1
94
1
4
6
6
3
1
2
103
1
2
4
42
1
16
5
2
2
6
1
1
1
22
1
1
1
1
1
1
1
1
1
2
31
2
1
5
3
7
1
1
1
1
1
38
1
1
3
1
19
1
7
2
1
1
3
1
1
1
1
1
1
1
1
2
1
2
1
2
1
3
3
3
1
1
1
2
1
1
1
1
1
4
2
1
1
8
1
15
3
1
5
1
1
3
4
1
1
2
4
11
45
1
1
6
4
1
2
1
1
1
2
16
1
1
1
1
1
3
1
30
1
2
8
1
12
4
3
1
1
2
1
1
1
3
2
1
5


35
6
1
9
1
1
1
13
2
4
4
1
7
1
1
1
10
1
5146
2
16
3
1
1
1
6
1
10
1
1
1
1
3
18
7
1
2
1
9
1
1
1
2
2
1
3
1
1
3
1
20
6
3
9
532
2
1
7
88
1
1
2
1
1
4
1
1
16
3
1
1
2
1
2
1
1
2
1
1
1
3
3
1
1
4
79
4
9
1
26
3
12
1
1
2
1
2
1
1
3
1
3
35
1
2
1
1
17
4
2
5
20
6
1
1
2
1
1
1
28
2
26
6
1
2
1
8
1
4
1
2
8
3
2
1
1
1
1
2
13
1
6
9
24
1
2
1
1
14
1
15
4
1
1
30
5
1
1
1
3
2
15
2
1
4
1
1
1
5
2
4
2
2
30
1
1
5
7
34
3
5
13
7
1
1
2
1
1
1
1
3
1
1
1
2
1
3
1
1
19
1
4
1
1
1
4
2
3
16
1
1
16
1
4
2
3
1
1
1
1
4
1
4
5
3
1
1
26
1
2
3
2
2
3
1
3
1
1
1
15
1
2
2
2
6
1
1
2
223
1
5
1
1
12
1
13
6
4
4
1
1
74
2
1
2
2
1
1
1
4
14
1
1
1
9
2
3
3
72
5
2
1
37
60
1
55
6
1
1
1
3
1
1
1
10
4
1
1
2
2
1
12
1
1
5
46
13
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
20
1
1
1
2
1
1
1
2
2
17
1
26
19
4
1
1
1
1
1
1
1
317
1
2
8
4
1
1
2
1
13
1
1
2
6
3
14
1
4
1
1
2
1
10
73
2
5
1
2
13
1
33
5
1
1
1
1
5
1
13
1
1
1
1
2
1
2
1
1
1
1
1
2
10
1
1
8
1
2
4
1
1
2
1
26
2
1
1
1
5
1
1
1
3
1
3
1
1
95
4
1
14
34
1
2
3
2
16
1
1
9
1
10
115
1
1
17
9
12
1
17
96
1
1
8
2
3
1
1
1
23
2
2
26
10
1

8
5
1
5
22
1
1
2
1
1
57
1
2
2
1
1
1
1
2
10
5
38
3
3
1
1
13
4
3
9
6
3
29
1
3
15
2
1
2
1
19
1
1
2
9
3
1
2
1
8
4
157
1
1
4
1
3
3
2
1
1
1
1
1
1
176
1
1
2
3
1
5
1
7
13
2
1
4
7
2
1
1
2
8
1
1
8
2
2
122
7
1
3
1
1
1
2
1
1
19
2
1
2
5
1
2
1
1
3
3
1
1
20
1
51
1
1
1
1
2
14
1
1
1
1
1
4
2
1
6
1
4
6
1
3
3
2
1
5
2
2
1
1
2
1
2
3
1
1
1
1
4
28
1
1
12
1
1
1
154
1
5
3
7
2
1
1
12
1
7
17
2
1
5
1
4
1
2
1
1
7
13
1
1
1
1
1
13
86
5
1
4
1
3
1
1
15
1
2
1
2
25
1
1
1
1
1
1
1
2
5
4
1
2
3
20
5
1
9
1
1
1
9
3
5
15
4
1
2
1
4
3
17
1
4
1
39
5
4
2
48
3
1
1
4
1
62
12
1
4
1
1
2
1
2
20
5
5
1
4
4
1
1
3
1
7
1
1
13
1
1
2
2
77
1
1
8
1
1
3
1
1
1
1
2
12
98
2
1
3
18
1
3
1
8
1
1
14
1
3
1
11
7
51
3
8
8
1
2
3
8
1
1
1
3
3
1
1
2
1
1
2
13
1
4
1
2
1
66
4
33
1
19
13
1
1
17
2
4
7
1
1
6
1
2
1
50
7
2
1
3
3
1
9
1
2
3
1
1
1
5
1
2
3
1
1
1
11784
9
1
2
1
1
2
67
1
3
1
1
1
1
17
1
2345
47
9
3
13
4
1
4
1
1
5
1
7
17
2
39
4
1
1
6
2
2
1
1
17
1
1
4
2
1
2
9
1
1
1
1
1
10
1
2
2
1
5
3
1
2
1
10
8
56
4
1
1
2
6
29
1
29
2
2
1
3
8
4
1
1
1
1
31
10
1
1
1
11
1
16
2
1
3


1
1
1
3
7
5
1
2
5
1
1
1
2
6
95
1
1
4
5
2
2
3
1
1
2
3
1
2
6
13
7
59
5
1
2
1
1
1
1
2
1
1
1
1
14
3
1
13
1
33
2
1
5
31
2
1
6
1
10
1
2
1
1
6
1
1
2
10
1
1
1
36
5
4
3
15
25
1
1
5
7
1
9
1
3
5
5
1
1
1
2
43
1
4
56
1
1
3
1
1
5
10
1
1
3
3
1
1
1
4
1
3
1
18
2
1
54
2
1
6
1
2
61
2
1
6
5
1
4
1
1
1
1
2
2
1
1
1
3
1
1
6
2
4
2
5
2
1
1
1
48
3
1
1
1
5
1
5
4
1
10
1
4
1
1
4
3
2
1
4
10
6
1
1
1
4
3
1
127
3
6
2
1
3
5
1
3
1
5
14
2
1
1
3
1
1
4
5
1
2
1
3
3
20
1
1
5
1
2
1
1
2
1
1
3
1
4
2
2
1
1
3
1
6
1
2
1
2
1
1
9
22
2
1
103
1
3
3
1
1
3
5
5
5
2
3
7
5
1
2
3
23
3
1
1
1
5
11
1
1
2
1
1
181
1
2
9
2
1
3
18
1
10
8
2
1
3
1
2
7
2
1
1
9
2
1
2
2
1
6
2
1
1
1
6
6
1
2
1
1
31
2
96
10
1
3
1
1
1
1
2
2
3
2
1
11
11
1
1
1
2
2
1
3
1
1
8
1
1
4
1
1
2
1
2
8
1
3
4
4
73
1
1
2
3
3
3
1
49
33
4
1
1
7
7
4
1
1
4
1
3
2
47
1
1
1
1
9
5
1
31
3
1
2
10
15
1
3
1
1
1
1
19
27
4
1
1
1
3
4
1
1
24
1
2
2
1
1
3
1
9
4
1
1
11
1
89
1
3
1
1
17
1
3
1
3
10
1
12
1
8
7
1
1
1
2
1
1
2
2
1
1
1
1
5
14
129
1
8
70
1
70
2
4
2
1
5
1
1
1
4
1
1
2
4
55
2
1
4
5
8
1
5
2
1
4
1
1
3
1


In [ ]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)
basic_model.add_states()

# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions
basic_model.add_transition()


# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [20]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

In [145]:
# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)

emission_probability = {}

basic_model.add_states()

In [ ]:
prob_emission = {}
states = {}
for tag, word_counts in emission_counts.items():
    prob_emission = {word : word_count/sum(word_counts.values()) for word, word_count in word_counts.items()}
    states[tag] = State(DiscreteDistribution(prob_emission), name=tag)

unique_tags = list(data.training_set.tagset)
for tag in unique_tags:
    basic_model.add_states(states[tag])

In [143]:
a=word_counts.values()

In [67]:
#for word, word_countinfo in word_counts.items():
 #   print(word_countinfo.values())

In [51]:
a=emission_counts.items()

In [144]:
b=word_counts.items()

In [ ]:
# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions
basic_model.add_transition()

In [ ]:
# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

In [21]:
from pomegranate import *
s1 = State(NormalDistribution(5, 1))
s2 = State(NormalDistribution(1, 7))
s3 = State(NormalDistribution(8, 2))
model = HiddenMarkovModel()
model.add_states(s1, s2, s3)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s1, 0.7)
model.add_transition(s1, s2, 0.3)
model.add_transition(s2, s2, 0.8)
model.add_transition(s2, s3, 0.2)
model.add_transition(s3, s3, 0.9)
model.add_transition(s3, model.end, 0.1)
model.bake()

In [ ]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

### Example Decoding Sequences with the HMM Tagger

In [ ]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [ ]:
!!jupyter nbconvert *.ipynb

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]